In [1]:
import os
import json
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
SIMULATION_DIRECTORY_1 = 'data/experiment_run_2'

In [3]:
reservoir_dynamics = []
entangled_labels = []

for filename in os.listdir(SIMULATION_DIRECTORY_1):
    if filename.endswith('.json'):
        filepath = os.path.join(SIMULATION_DIRECTORY_1, filename)
        with open(filepath, 'r') as file:
            json_data = json.load(file)
            reservoir_dynamics.append(json_data['reservoir_dynamics'])
            entangled_labels.append(json_data['entangled_label'])

In [4]:
reservoir_dynamics = np.array(reservoir_dynamics)
entangled_labels = np.array(entangled_labels)
entangled_labels = entangled_labels.reshape(entangled_labels.shape[0], 1)

print(f'Shape of reservoir dyanmics is {reservoir_dynamics.shape}')
print(f'Shape of entangled labels is {entangled_labels.shape}')

Shape of reservoir dyanmics is (10000, 5, 321)
Shape of entangled labels is (10000, 1)


In [5]:
n_datapoints = reservoir_dynamics.shape[0]
n_reservoir_qubits = reservoir_dynamics.shape[1]
n_timepoints = reservoir_dynamics.shape[2]

In [6]:
########## Dimensionality calculation ############
# We have to avoid the curse of dimensionality so will go with the 
# 10:1 ratio of datapoints:features for now
# TODO: Tune this as a hyperparameter
DIMENSIONALITY_RATIO = 50

required_features = int(n_datapoints // DIMENSIONALITY_RATIO)

allowed_n_timepoints = 50

# Calculate the indices to access from the original array 
time_sample_indices = np.linspace(0, n_reservoir_qubits - 1, allowed_n_timepoints, dtype=int)

sampled_reservoir_dynamics = reservoir_dynamics[:, :, time_sample_indices]

print(f"The new sampled reservoir dynmaics shape is {sampled_reservoir_dynamics.shape}")

The new sampled reservoir dynmaics shape is (10000, 5, 50)


In [7]:
# Flatten the training data column-wise
# This ensures that all readings of each particle in the reservoir 
# at a given time point are adjacent to each other in the data.
# Transpose the last two dimensions to prepare for column-wise flattening
transponsed_sampled_reservoir_dynamics = sampled_reservoir_dynamics.transpose(0, 2, 1)  

# Flatten to (n_datapoints, n_features)
reservoir_dynamics_flattened = transponsed_sampled_reservoir_dynamics.reshape(n_datapoints, -1)

print("The training data has been flattened and is now shape", reservoir_dynamics_flattened.shape)

The training data has been flattened and is now shape (10000, 250)


In [8]:
# Train the random forest!
X_train, X_test, y_train, y_test = train_test_split(reservoir_dynamics_flattened, entangled_labels, test_size=0.2, random_state=42, shuffle=True)

In [19]:
rf_classifier = RandomForestClassifier(
    # n_estimators=100,
    # max_depth=7, 
    # min_samples_split=10,  
    # min_samples_leaf=5,
    random_state=42,
    )

In [20]:
# Step 4: Perform cross-validation on the training data
cv_scores = cross_val_score(rf_classifier, reservoir_dynamics_flattened, entangled_labels, cv=5)

print("Cross-validation scores:", cv_scores)
print("Average cross-validation score:", cv_scores.mean())

/Users/irvinc01/10env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/irvinc01/10env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/irvinc01/10env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/irvinc01/10env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConvers

Cross-validation scores: [0.581  0.576  0.5765 0.567  0.5645]
Average cross-validation score: 0.5730000000000001


In [21]:
rf_classifier.fit(X_train, y_train)

/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_71676/31409248.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [22]:
y_pred_train = rf_classifier.predict(X_train)
y_pred = rf_classifier.predict(X_test)

In [23]:
# Evaluate the model
train_accuracy = accuracy_score(y_train, y_pred_train)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Train Accuracy: 0.9955
Test Accuracy: 0.57
Confusion Matrix:
 [[560 428]
 [432 580]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.56      0.57      0.57       988
         1.0       0.58      0.57      0.57      1012

    accuracy                           0.57      2000
   macro avg       0.57      0.57      0.57      2000
weighted avg       0.57      0.57      0.57      2000

